In [1]:
import sys
import os
import random
# Counter
from collections import Counter
# Package
import somhos.resources.dataset as rd
import somhos.resources.queries as rq
from somhos.methods.useful import save_pickle, load_pickle, wordvectors_centroid
from somhos.config.paths import *

Default path

In [2]:
prefix_path = "../../"
data_path = get_relative_path(prefix_path, V9GAMMA_PATH)
os.path.exists(data_path)

True

Load samples

In [3]:
# Load sample A
docs_sample_a_path = get_relative_path(data_path, DOCS_SAMPLE_A_SUFFIX)
docs_sample_a = load_pickle(docs_sample_a_path)
# Load sample B
docs_sample_b_path = get_relative_path(data_path, DOCS_SAMPLE_B_SUFFIX)
docs_sample_b = load_pickle(docs_sample_b_path)

print("Samples size: (%d, %d)" % (len(docs_sample_a), len(docs_sample_b)))

Samples size: (5000, 5000)


Load samples content

In [4]:
docs_samples_content_path = get_relative_path(data_path, DOCS_SAMPLES_CONTENT)
samples_content = load_pickle(docs_samples_content_path)
print("Docs: %d" % len(samples_content))

Docs: 10000


In [5]:
import numpy as np
from scipy.spatial.distance import cosine as cosine_distance

Load counts documents by keyprase

In [6]:
docs_samples_kps_count_path = get_relative_path(data_path, DOCS_SAMPLES_KPS_COUNT)
samples_kps_count = load_pickle(docs_samples_kps_count_path)
Vlen = len(samples_kps_count['count'])

In [7]:
sample_a_indices = range(0, len(docs_sample_a))
sample_b_indices = range(0, len(docs_sample_b))

def get_content_for(index, samples_content, field='kps-normalized'):
    return samples_content[index][field]

def tf_doc(terms):
    tf = Counter(terms)
    cnt = sum(tf.values())
    return {k: v/cnt for k, v in tf.items()}

m_measure_sim = np.zeros([len(docs_sample_a), len(docs_sample_b)])
for i in range(0, len(sample_a_indices)):
    # Term frequencies by document
    tf_a_dict = tf_doc(get_content_for(docs_sample_a[i], samples_content))
    for j in range(0, len(sample_b_indices)):
        # Term frequencies by document
        tf_b_dict = tf_doc(get_content_for(docs_sample_b[j], samples_content))
        # Term list
        t_list = list(set(tf_a_dict) | set(tf_b_dict))
        # TF
        tf_a = np.zeros([len(t_list),])
        tf_b = np.zeros([len(t_list),])  
        for pos, t in enumerate(t_list):
            if t in tf_a_dict:
                tf_a[pos] = tf_a_dict[t]
            if t in tf_b_dict:
                tf_b[pos] = tf_b_dict[t]
        # Measure
        measure_sim = 1.0 - cosine_distance(tf_a, tf_b)
        if measure_sim != 0:
            m_measure_sim[i, j] = measure_sim
print(m_measure_sim)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.02326211 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.02326211 0.         0.        ]]


In [9]:
pickle_path = get_relative_path(data_path, DOCS_SAMPLES_TF_KPS)
save_pickle(m_measure_sim, pickle_path)